**Задача 1: Система управления библиотекой**

Создайте систему управления библиотекой используя collections:

1. Создайте namedtuple `Book` с полями: title, author, isbn, year
2. Создайте namedtuple `Reader` с полями: name, reader_id, phone
3. Используйте `defaultdict(list)` для хранения книг по жанрам
4. Используйте `deque` для очереди читателей, ожидающих популярную книгу
5. Используйте `Counter` для подсчета количества книг каждого автора
6. Используйте `OrderedDict` для хранения истории выдачи книг (читатель -> список книг)
7. Сериализуйте все данные в JSON и pickle форматы

In [11]:
from collections import namedtuple, defaultdict, deque, Counter, OrderedDict
import json
import pickle
from typing import Dict, List, Deque, Any

Book = namedtuple('Book', ['title', 'author', 'isbn', 'year'])
Reader = namedtuple('Reader', ['name', 'reader_id', 'phone'])

class LibrarySystem:
    """Система управления библиотекой"""
    
    def __init__(self) -> None:
        self.books_by_genre: Dict[str, List[Book]] = defaultdict(list)
        self.waiting_queue: Deque[Reader] = deque()
        self.author_counter: Counter = Counter()
        self.history: OrderedDict[Reader, List[Book]] = OrderedDict()

    def add_book(self, book: Book, genre: str) -> None:
        """Добавляет книгу в систему по указанному жанру"""
        self.books_by_genre[genre].append(book)
        self.author_counter[book.author] += 1

    def add_reader_to_queue(self, reader: Reader) -> None:
        """Добавляет читателя в очередь ожидания"""
        self.waiting_queue.append(reader)

    def lend_book(self, reader: Reader, book: Book) -> None:
        """Регистрирует выдачу книги читателю"""
        if reader not in self.history:
            self.history[reader] = []
        self.history[reader].append(book)

    def to_dict(self) -> Dict[str, Any]:
        """Преобразует все данные в словари для сериализации"""
        books_dict = {}
        for genre, books in self.books_by_genre.items():
            books_dict[genre] = [book._asdict() for book in books]

        queue_list = [reader._asdict() for reader in self.waiting_queue]

        counter_dict = dict(self.author_counter)
        

        history_dict = {}
        for reader, books in self.history.items():
            history_dict[reader.reader_id] = {
                'reader': reader._asdict(),
                'books': [book._asdict() for book in books]
            }
        
        return {
            'books_by_genre': books_dict,
            'waiting_queue': queue_list,
            'author_counter': counter_dict,
            'history': history_dict
        }

    def save_json(self, filename: str) -> None:
        """Сохраняет все данные в JSON файл"""
        data = self.to_dict()
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4, ensure_ascii=False)

    def save_pickle(self, filename: str) -> None:
        """Сохраняет всю систему в pickle файл"""
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load_pickle(cls, filename: str) -> 'LibrarySystem':
        """Загружает систему из pickle файла"""
        with open(filename, 'rb') as f:
            return pickle.load(f)

if __name__ == "__main__":
    library = LibrarySystem()

    book1 = Book("1984", "Джордж Оруэлл", "123-456", 1949)
    book2 = Book("О дивный новый мир", "Олдос Хаксли", "789-012", 1932)
    book3 = Book("Мастер и Маргарита", "Михаил Булгаков", "345-678", 1967)

    library.add_book(book1, "Антиутопия")
    library.add_book(book2, "Антиутопия")
    library.add_book(book3, "Роман")

    reader1 = Reader("Иван Иванов", "001", "+79161234567")
    reader2 = Reader("Петр Петров", "002", "+79167654321")

    library.add_reader_to_queue(reader1)
    library.add_reader_to_queue(reader2)

    library.lend_book(reader1, book1)
    library.lend_book(reader2, book2)
    library.lend_book(reader1, book3)

    library.save_json("library_data.json")
    library.save_pickle("library_data.pkl")

    print("Книги по жанрам:")
    for genre, books in library.books_by_genre.items():
        print(f"{genre}: {len(books)} книг(и)")

    print("\nОчередь читателей:")
    for i, reader in enumerate(library.waiting_queue, 1):
        print(f"{i}. {reader.name}")

    print("\nСтатистика авторов:")
    for author, count in library.author_counter.items():
        print(f"{author}: {count} книг(и)")

    print("\nИстория выдачи:")
    for reader, books in library.history.items():
        print(f"{reader.name}: {len(books)} книг(и)")

Книги по жанрам:
Антиутопия: 2 книг(и)
Роман: 1 книг(и)

Очередь читателей:
1. Иван Иванов
2. Петр Петров

Статистика авторов:
Джордж Оруэлл: 1 книг(и)
Олдос Хаксли: 1 книг(и)
Михаил Булгаков: 1 книг(и)

История выдачи:
Иван Иванов: 2 книг(и)
Петр Петров: 1 книг(и)


In [ ]:
#Реализуйте простую модель стоянки. 
# 1) Скажем, ваши автомобили это именованные кортежи, с указанием марки автомобиля, модели, гос.номера и цвета
# 2) Сама стоянка это двусторонняя очередь (deque)
#   Представим, что у нас нет коллизий, и если места на стоянке нет, то виртуально вызывается эвакуатор и убирает самый крайний автомобиль (как и работает deque)
# 3) Создайте набор автомобилей и в случайном порядке добавляйте/удаляйте их с автостоянки.
# 4) Посчитайте, сколько и каких автомобилей осталось на автостоянке.

In [1]:
from collections import deque, namedtuple
import random
from typing import List, Dict

# 1) Создаем именованный кортеж для автомобилей
Car = namedtuple('Car', ['brand', 'model', 'license_plate', 'color'])

class ParkingLot:
    """Класс автостоянки с ограниченным количеством мест"""
    
    def __init__(self, capacity: int = 5):
        self.capacity = capacity
        self.parking = deque(maxlen=capacity)  # 2) Двусторонняя очередь с ограничением
        self.removed_cars = []  # Для учета эвакуированных автомобилей
    
    def park_car(self, car: Car) -> str:
        """Добавить автомобиль на стоянку"""
        if len(self.parking) < self.capacity:
            self.parking.append(car)
            return f"{car.brand} {car.model} припаркован на свободное место"
        else:
            # 2) Если нет мест - эвакуируем самый старый автомобиль
            removed_car = self.parking.popleft()
            self.removed_cars.append(removed_car)
            self.parking.append(car)
            return f"Эвакуирован {removed_car.brand} {removed_car.model}, припаркован {car.brand} {car.model}"
    
    def remove_car(self, license_plate: str) -> str:
        """Удалить автомобиль по номеру"""
        for i, car in enumerate(self.parking):
            if car.license_plate == license_plate:
                self.parking.remove(car)
                return f"{car.brand} {car.model} уехал со стоянки"
        return f"Автомобиль с номером {license_plate} не найден на стоянке"
    
    def get_parking_stats(self) -> Dict[str, int]:
        """Статистика по автомобилям на стоянке"""
        stats = {}
        for car in self.parking:
            brand = car.brand
            stats[brand] = stats.get(brand, 0) + 1
        return stats
    
    def display_parking(self) -> str:
        """Визуальное отображение стоянки"""
        if not self.parking:
            return " Стоянка пуста"
        
        result = " Текущее состояние стоянки:\n"
        for i, car in enumerate(self.parking, 1):
            result += f"{i}. {car.color} {car.brand} {car.model} [{car.license_plate}]\n"
        return result
    
    def get_evacuated_stats(self) -> Dict[str, int]:
        """Статистика по эвакуированным автомобилям"""
        stats = {}
        for car in self.removed_cars:
            brand = car.brand
            stats[brand] = stats.get(brand, 0) + 1
        return stats

# 3) Создаем набор автомобилей
def generate_cars(count: int = 20) -> List[Car]:
    """Генерация случайных автомобилей"""
    brands_models = {
        'Toyota': ['Camry', 'Corolla', 'RAV4', 'Prius'],
        'Honda': ['Civic', 'Accord', 'CR-V', 'Pilot'],
        'BMW': ['X5', '3 Series', '5 Series', 'X3'],
        'Mercedes': ['C-Class', 'E-Class', 'GLC', 'S-Class'],
        'Audi': ['A4', 'A6', 'Q5', 'Q7'],
        'Hyundai': ['Solaris', 'Elantra', 'Tucson', 'Santa Fe'],
        'Kia': ['Rio', 'Optima', 'Sportage', 'Sorento']
    }
    
    colors = ['Красный', 'Синий', 'Зеленый', 'Черный', 'Белый', 'Серый', 'Желтый']
    
    cars = []
    for i in range(count):
        brand = random.choice(list(brands_models.keys()))
        model = random.choice(brands_models[brand])
        license_plate = f"{random.choice(['A','B','C'])}{random.randint(100, 999)}{random.choice(['AB','CD','EF'])}"
        color = random.choice(colors)
        
        cars.append(Car(brand, model, license_plate, color))
    
    return cars

def main():
    """Основная функция демонстрации работы стоянки"""
    print("=== МОДЕЛЬ АВТОСТОЯНКИ ===\n")
    
    # Создаем стоянку на 5 мест
    parking = ParkingLot(capacity=5)
    
    # Генерируем 15 автомобилей
    cars = generate_cars(15)
    
    print("Сгенерированные автомобили:")
    for i, car in enumerate(cars, 1):
        print(f"{i}. {car.color} {car.brand} {car.model} [{car.license_plate}]")
    
    print(f"\n Вместимость стоянки: {parking.capacity} мест\n")
    
    # 3) Случайные операции с автомобилями
    operations = []
    for i in range(20):
        if random.random() < 0.6 or len(parking.parking) == 0:  # 60% шанс на парковку
            if cars:  # Если есть автомобили для парковки
                car = random.choice(cars)
                cars.remove(car)
                result = parking.park_car(car)
                operations.append(f"Попытка парковки: {result}")
        else:  # 40% шанс на выезд
            if parking.parking:  # Если есть автомобили на стоянке
                car_to_remove = random.choice(list(parking.parking))
                result = parking.remove_car(car_to_remove.license_plate)
                cars.append(car_to_remove)  # Возвращаем автомобиль в общий список
                operations.append(f"Попытка выезда: {result}")
    
      # Выводим историю операций
    print("История операций:")
    for i, op in enumerate(operations[-10:], 1):  # Показываем последние 10 операций
        print(f"{i}. {op}")
    
    print("\n" + "="*50)
    
    # 4) Финальная статистика
    print(parking.display_parking())
    
    # Статистика по оставшимся автомобилям
    current_stats = parking.get_parking_stats()
    print("\n Статистика автомобилей на стоянке:")
    for brand, count in current_stats.items():
        print(f"   {brand}: {count} автомобилей")
    
    # Статистика по эвакуированным автомобилям
    evacuated_stats = parking.get_evacuated_stats()
    if evacuated_stats:
        print("\n Статистика эвакуированных автомобилей:")
        for brand, count in evacuated_stats.items():
            print(f"   {brand}: {count} автомобилей")
        print(f"   Всего эвакуировано: {len(parking.removed_cars)} автомобилей")
    else:
        print("\n Ни один автомобиль не был эвакуирован")

if __name__ == "__main__":
    main()

=== МОДЕЛЬ АВТОСТОЯНКИ ===

Сгенерированные автомобили:
1. Желтый Hyundai Solaris [C176CD]
2. Зеленый Hyundai Tucson [C389EF]
3. Синий Audi A4 [A752EF]
4. Синий Audi Q5 [A408EF]
5. Зеленый Toyota RAV4 [B793CD]
6. Красный Mercedes S-Class [A873CD]
7. Красный Kia Rio [C273CD]
8. Желтый BMW 3 Series [C106EF]
9. Серый Hyundai Solaris [B420AB]
10. Черный Mercedes GLC [A375CD]
11. Черный Mercedes S-Class [C617AB]
12. Красный BMW X3 [C839EF]
13. Синий Audi Q5 [B380CD]
14. Белый BMW X3 [A478EF]
15. Черный BMW 5 Series [A552EF]

 Вместимость стоянки: 5 мест

История операций:
1. Попытка парковки: Audi Q5 припаркован на свободное место
2. Попытка парковки: Toyota RAV4 припаркован на свободное место
3. Попытка выезда: Audi Q5 уехал со стоянки
4. Попытка парковки: Audi Q5 припаркован на свободное место
5. Попытка выезда: Audi Q5 уехал со стоянки
6. Попытка парковки: Hyundai Solaris припаркован на свободное место
7. Попытка парковки: BMW 5 Series припаркован на свободное место
8. Попытка выезда: Hy

**Задача 2: Анализатор файловой системы**

Создайте анализатор файловой системы используя os и sys:

1. Создайте namedtuple `FileInfo` с полями: name, size, extension, modified_time
2. Используйте `os.walk()` для обхода директории
3. Используйте `os.path` функции для получения информации о файлах
4. Используйте `Counter` для подсчета файлов по расширениям
5. Используйте `defaultdict(list)` для группировки файлов по размеру (маленькие < 1MB, средние 1-100MB, большие > 100MB)
6. Используйте `deque` для хранения последних 10 найденных файлов
7. Выведите статистику используя `sys.getsizeof()` для подсчета памяти
8. Сохраните результаты в JSON файл


**Задача 3: Система конфигурации приложения**

Создайте систему конфигурации используя ChainMap и defaultdict:

1. Создайте namedtuple `Config` с полями: key, value, section, default_value
2. Создайте несколько словарей конфигурации (default, user, environment)
3. Используйте `ChainMap` для объединения конфигураций с приоритетом
4. Используйте `defaultdict(dict)` для группировки настроек по секциям
5. Используйте `OrderedDict` для сохранения порядка загрузки конфигураций
6. Используйте `os.environ` для чтения переменных окружения
7. Сериализуйте конфигурацию в JSON и pickle форматы

**Задача 4: Мониторинг системы**

Создайте систему мониторинга используя sys и os:

1. Создайте namedtuple `SystemInfo` с полями: cpu_count, memory_usage, process_id, user_name
2. Используйте `os.cpu_count()` для получения количества процессоров
3. Используйте `sys.getallocatedblocks()` для мониторинга памяти
4. Используйте `os.getpid()` и `os.getlogin()` для информации о процессе
5. Используйте `deque` для хранения последних 20 измерений
6. Используйте `Counter` для подсчета частоты использования различных функций
7. Используйте `defaultdict(list)` для группировки измерений по времени
8. Сохраните историю мониторинга в pickle файл

**Задача 5: Система логирования**

Создайте систему логирования используя все изученные коллекции:

1. Создайте namedtuple `LogEntry` с полями: timestamp, level, message, module, function
2. Используйте `deque` для хранения последних 100 логов (кольцевой буфер)
3. Используйте `defaultdict(list)` для группировки логов по уровням (DEBUG, INFO, WARNING, ERROR)
4. Используйте `Counter` для подсчета количества логов каждого уровня
5. Используйте `OrderedDict` для хранения логов по времени (FIFO)
6. Используйте `ChainMap` для объединения различных источников логов
7. Используйте `os.path` для работы с файлами логов
8. Сериализуйте логи в JSON и pickle форматы

**Задача 6: Кэш-система**

Создайте простую кэш-систему используя collections:

1. Создайте namedtuple `CacheEntry` с полями: key, value, timestamp, access_count
2. Используйте `OrderedDict` для реализации LRU (Least Recently Used) кэша
3. Используйте `deque` для хранения истории доступа к ключам
4. Используйте `Counter` для подсчета частоты доступа к каждому ключу
5. Используйте `defaultdict(int)` для хранения счетчиков доступа
6. Реализуйте методы: get, set, delete, clear, size
7. Используйте `sys.getsizeof()` для мониторинга размера кэша
8. Сериализуйте кэш в pickle формат для сохранения между сессиями


**Задача 7: Анализатор текста**

Создайте анализатор текста используя collections:

1. Создайте namedtuple `WordInfo` с полями: word, frequency, length, first_occurrence
2. Используйте `Counter` для подсчета частоты слов
3. Используйте `defaultdict(list)` для группировки слов по длине
4. Используйте `deque` для хранения последних 50 уникальных слов
5. Используйте `OrderedDict` для хранения слов в порядке первого появления
6. Используйте `os.path` для работы с текстовыми файлами
7. Используйте `sys.getsizeof()` для анализа памяти
8. Сохраните результаты анализа в JSON файл